<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.2.1
# *Querying the International Space Station*

## The OpenNotify API

The OpenNotify API exposes a few attributes of the International Space Station (ISS) via a simple, authentication-free interface. The simplicity of this API precludes any need for a dedicated Python library. However, as with many APIs, it accepts requests according to HTTP standards and returns responses in JSON format, so the Python libraries request and json will make managing the I/O simpler still.

In [5]:
import requests
import json
from datetime import datetime, date, time

This request fetches the latest position of the international space station:

In [7]:
response = requests.get("http://api.open-notify.org/iss-now.json")

Print the status code and text of the response:

In [9]:
#ANSWER
response.status_code

200

In [10]:
#ANSWER
print(response.text)

{"iss_position": {"latitude": "29.7769", "longitude": "-145.4820"}, "timestamp": 1742889090, "message": "success"}


We can use another API to request the current position of the ISS and the next few times at which it will be over a certain location. The latitude and longitude of Sydney are (-33.87, 151.21).

In [12]:
response = requests.get("https://api.g7vrd.co.uk/v1/satellite-passes/25544/-33.87/151.21.json?minelevation=0&hours=24")

Print the response header:

In [14]:
#ANSWER
print(response.headers)

{'Date': 'Tue, 25 Mar 2025 07:51:32 GMT', 'Server': 'Apache', 'X-Amzn-Trace-Id': 'Root=1-67e26084-f00693d98f9926fbdada9462;', 'Vary': 'Origin,Access-Control-Request-Method,Access-Control-Request-Headers', 'Access-Control-Allow-Origin': '*', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '0', 'Cache-Control': 'no-cache, no-store, max-age=0, must-revalidate', 'Pragma': 'no-cache', 'Expires': '0', 'X-Frame-Options': 'DENY', 'Content-Type': 'application/json', 'Keep-Alive': 'timeout=5, max=100', 'Connection': 'Keep-Alive', 'Transfer-Encoding': 'chunked'}


Print the content of the response (the data that the server returned):

In [16]:
#ANSWER
print(response.content)

b'{"api_status":"ALPHA","request_timestamp":"2025-03-25T07:51:32.143942226Z","norad_id":25544,"satellite_name":"ISS","tle_last_retrieved":"2025-03-24T17:43:36.947922442Z","lat":-33.87,"lon":151.21,"hours":24,"min_elevation":0,"query_ms":17,"passes":[{"start":"2025-03-25T14:17:37.127Z","tca":"2025-03-25T14:21:37.127Z","end":"2025-03-25T14:25:32.127Z","aos_azimuth":9,"los_azimuth":103,"max_elevation":8.0},{"start":"2025-03-25T15:52:17.127Z","tca":"2025-03-25T15:57:47.127Z","end":"2025-03-25T16:03:17.127Z","aos_azimuth":311,"los_azimuth":134,"max_elevation":77.0},{"start":"2025-03-25T17:30:27.127Z","tca":"2025-03-25T17:34:57.127Z","end":"2025-03-25T17:39:37.127Z","aos_azimuth":262,"los_azimuth":149,"max_elevation":11.0},{"start":"2025-03-25T19:10:12.127Z","tca":"2025-03-25T19:13:12.127Z","end":"2025-03-25T19:16:12.127Z","aos_azimuth":219,"los_azimuth":152,"max_elevation":3.0},{"start":"2025-03-25T20:48:07.127Z","tca":"2025-03-25T20:51:37.127Z","end":"2025-03-25T20:55:17.127Z","aos_azimuth

Note that this is a Python byte string:

In [18]:
print(type(response.content))

<class 'bytes'>


Print just the "content-type" value from the header:

In [20]:
#ANSWER
print(response.headers['content-type'])

application/json


JSON was designed to be easy for computers to read, not for people. The `requests` library can decode the JSON byte string:

In [22]:
overheads = response.json()
print(overheads)

{'api_status': 'ALPHA', 'request_timestamp': '2025-03-25T07:51:32.143942226Z', 'norad_id': 25544, 'satellite_name': 'ISS', 'tle_last_retrieved': '2025-03-24T17:43:36.947922442Z', 'lat': -33.87, 'lon': 151.21, 'hours': 24, 'min_elevation': 0, 'query_ms': 17, 'passes': [{'start': '2025-03-25T14:17:37.127Z', 'tca': '2025-03-25T14:21:37.127Z', 'end': '2025-03-25T14:25:32.127Z', 'aos_azimuth': 9, 'los_azimuth': 103, 'max_elevation': 8.0}, {'start': '2025-03-25T15:52:17.127Z', 'tca': '2025-03-25T15:57:47.127Z', 'end': '2025-03-25T16:03:17.127Z', 'aos_azimuth': 311, 'los_azimuth': 134, 'max_elevation': 77.0}, {'start': '2025-03-25T17:30:27.127Z', 'tca': '2025-03-25T17:34:57.127Z', 'end': '2025-03-25T17:39:37.127Z', 'aos_azimuth': 262, 'los_azimuth': 149, 'max_elevation': 11.0}, {'start': '2025-03-25T19:10:12.127Z', 'tca': '2025-03-25T19:13:12.127Z', 'end': '2025-03-25T19:16:12.127Z', 'aos_azimuth': 219, 'los_azimuth': 152, 'max_elevation': 3.0}, {'start': '2025-03-25T20:48:07.127Z', 'tca': '2

What kind of object did this give us?

In [24]:
#ANSWER:
print(type(overheads))

<class 'dict'>


Python dicts are easier to work with, but the data we want is still buried in that data structure, so we have to dig it out. First, extract the `passes` value to a separate list:

In [26]:
#ANSWER:
passes = overheads['passes']

Now extract the `start` strings into an array called `srisetimes`:

In [28]:
#ANSWER:
srisetimes = [xpass['start'] for xpass in passes]

These are strings. We convert these to an array of Python `datetime` values called `risetimes`:

In [30]:
srisetimes = [datetime.strptime(xpass['start'], "%Y-%m-%dT%H:%M:%S.%fZ") for xpass in passes]

Finally, use `risetime.strftime` to print these in a format that people understand:

```
e.g.
18/10/22 07:05
18/10/22 08:41
18/10/22 10:20
18/10/22 12:00
18/10/22 01:37
18/10/22 03:13
```



In [32]:
#ANSWER:
risetimes = [datetime.strptime(xpass['start'], "%Y-%m-%dT%H:%M:%S.%fZ") for xpass in passes]
risetimes

[datetime.datetime(2025, 3, 25, 14, 17, 37, 127000),
 datetime.datetime(2025, 3, 25, 15, 52, 17, 127000),
 datetime.datetime(2025, 3, 25, 17, 30, 27, 127000),
 datetime.datetime(2025, 3, 25, 19, 10, 12, 127000),
 datetime.datetime(2025, 3, 25, 20, 48, 7, 127000),
 datetime.datetime(2025, 3, 25, 22, 24, 22, 127000),
 datetime.datetime(2025, 3, 26, 0, 1, 2, 127000),
 datetime.datetime(2025, 3, 26, 13, 31, 2, 127000)]

Finally, here is an endpoint that tells us who is on board:

In [34]:
response = requests.get("http://api.open-notify.org/astros.json")

Referring to the methods used above, extract the number of astronauts and their names:

In [36]:
#ANSWER:
astros = response.json()
print(astros)
print(astros["number"])
for astronaut in astros['people']:
    print(astronaut['name'])

{'people': [{'craft': 'ISS', 'name': 'Oleg Kononenko'}, {'craft': 'ISS', 'name': 'Nikolai Chub'}, {'craft': 'ISS', 'name': 'Tracy Caldwell Dyson'}, {'craft': 'ISS', 'name': 'Matthew Dominick'}, {'craft': 'ISS', 'name': 'Michael Barratt'}, {'craft': 'ISS', 'name': 'Jeanette Epps'}, {'craft': 'ISS', 'name': 'Alexander Grebenkin'}, {'craft': 'ISS', 'name': 'Butch Wilmore'}, {'craft': 'ISS', 'name': 'Sunita Williams'}, {'craft': 'Tiangong', 'name': 'Li Guangsu'}, {'craft': 'Tiangong', 'name': 'Li Cong'}, {'craft': 'Tiangong', 'name': 'Ye Guangfu'}], 'number': 12, 'message': 'success'}
12
Oleg Kononenko
Nikolai Chub
Tracy Caldwell Dyson
Matthew Dominick
Michael Barratt
Jeanette Epps
Alexander Grebenkin
Butch Wilmore
Sunita Williams
Li Guangsu
Li Cong
Ye Guangfu


## HOMEWORK


1. Write a simple handler for the response status code (refer to lab resources slide for HTTP response codes). As this Jupyter Notebook is an interactive device, the handler does not need to manage subsequent code execution (i.e. by branching or aborting execution), although it should return something that could be used to do so if deployed in a Python program.

In [39]:
#ANSWER:
def handleResponse(response, verbose = False):
    '''
    Returns Boolean Value, Status Code,
    '''
  # if Status Code is 200 return false, and status code
  # Otherwise Return True and Status Code
    if response.status_code == 200:
        if verbose:
            print('Response = 200: OK')
        fail = False
    else:
        if verbose:
            print('Response = ' + str(response.status_code) + ': ' + '(TODO: output of lookup function here)')
        fail = True
    return(fail, response.status_code)

2. Test your response handler on some correct and incorrect API calls.

In [41]:
#response = requests.get("http://api.open-notify.org/astros.json")
# Test incorrect API
response = requests.get("http://api.open-notify.org/astro.json")
if handleResponse(response)[0]:
    print('API call failed. Resolve issue before continuing!')

# Test correct API
response = requests.get("http://api.open-notify.org/iss-now.json")
handleResponse(response, True)[0]

API call failed. Resolve issue before continuing!
Response = 200: OK


False

>

>

>



---



---



> > > > > > > > > © 2025 Institute of Data


---



---



